In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
raw_data = pd.io.excel.read_excel('SzarugaMindMap_Draft_v2.xlsx',parse_cols='A:D')

In [3]:
raw_data.head()

,Source/Subject,Predicate,Target/Object,Weight (1-10)
0,Steve Szaruga,Knows,Hypersonic Weapons,9
1,Steve Szaruga,Belongs In,AFRL/RX,9
2,Hypersonic Weapons,Involves,Metals,9
3,Hypersonic Weapons,Involves,Soft Matter,1
4,Hypersonic Weapons,Involves,Ceramics,5


In [4]:
#boolean checks for whether a string is in a column
print sum(raw_data['Source/Subject'].str.contains('Steve Szaruga'))>0
print 'Steve Szaruga' in raw_data['Source/Subject'].values 

True
True


In [5]:
ontology = pd.io.excel.read_excel('SzarugaMindMap_Draft_v2.xlsx',parse_cols='F:J',skiprows=2)

In [6]:
ontology[:12] #appear to be only 30 unique terms in the ontology, 38 are in the raw_data

,Discliplines,RX Function,RXM Roadmap,Contact,Predicates
0,Hypersonic Weapons,Metals,Manufacturing Research,DARPA,Knows
1,Additive Manufacturing,Soft Matter,Structures,Andy Swanson,Belongs In
2,Aircraft Coatings,Ceramics,Propulsion-Turbine Engines,Ken Davidson,Involves
3,Stats/DOE,Composites,Air Armaments,Mary Kinsella,Is Studied By
4,Airframes,Data,Sustainment,RX,NaN
5,NaN,Process Control,LOM/LOA,RW,NaN
6,NaN,Simulation,C4ISR H/W-Aircraft,RQ,NaN
7,NaN,Inspection,C4ISR H/W-Space,NaN,NaN
8,NaN,Electronic Matls,NaN,NaN,NaN
9,NaN,Photonic Matls,NaN,NaN,NaN


In [7]:
ontology.keys()

Index([u'Discliplines', u'RX Function', u'RXM Roadmap', u'Contact', u'Predicates'], dtype='object')

In [8]:
#create a set of nodes named in Ontology list
from sets import Set
ont_set = Set()
for col in ontology.ix[:,:4]:
    col_set = Set(ontology[col].dropna().unique())
    ont_set.union_update(  col_set )

In [9]:
ont_set,len(ont_set)

(Set([u'Soft Matter', u'Manufacturing Research', u'Andy Swanson', u'DARPA', u'Simulation', u'Sustainment', u'RW', u'Additive Manufacturing', u'Mary Kinsella', u'Hypersonic Weapons', u'Ken Davidson', u'Composites', u'Inspection', u'Photonic Matls', u'LOM/LOA', u'RQ', u'Stats/DOE', u'Air Armaments', u'Aircraft Coatings', u'Propulsion-Turbine Engines', u'Structures', u'RX', u'Metals', u'C4ISR H/W-Space', u'Electronic Matls', u'C4ISR H/W-Aircraft', u'Process Control', u'Airframes', u'Data', u'Ceramics']),
 30)

In [10]:
#convert ontology from a pandas dataframe (table) to a key:value pair dictionary. 
#the key is the category, the value is the list of valid terms in that category
ontology = {key:ontology[key].dropna() for key in ontology.keys() }

In [11]:
ontology

{u'Contact': 0            DARPA
 1     Andy Swanson
 2     Ken Davidson
 3    Mary Kinsella
 4               RX
 5               RW
 6               RQ
 Name: Contact, dtype: object, u'Discliplines': 0        Hypersonic Weapons
 1    Additive Manufacturing
 2         Aircraft Coatings
 3                 Stats/DOE
 4                 Airframes
 Name: Discliplines, dtype: object, u'Predicates': 0            Knows
 1       Belongs In
 2         Involves
 3    Is Studied By
 Name: Predicates, dtype: object, u'RX Function': 0              Metals
 1         Soft Matter
 2            Ceramics
 3          Composites
 4                Data
 5     Process Control
 6          Simulation
 7          Inspection
 8    Electronic Matls
 9      Photonic Matls
 Name: RX Function, dtype: object, u'RXM Roadmap': 0        Manufacturing Research
 1                    Structures
 2    Propulsion-Turbine Engines
 3                 Air Armaments
 4                   Sustainment
 5                       LOM/LOA

In [12]:
#create a list of unique nodes
subject_nodes=raw_data['Source/Subject']
target_nodes = raw_data['Target/Object']
#combine the two nodes types and remove duplicates
nodes = subject_nodes.append(target_nodes).drop_duplicates()
#find out how many unique nodes will be added to the graph
nodes.shape

(38,)

In [13]:
nodes

0                   Steve Szaruga
2              Hypersonic Weapons
13         Additive Manufacturing
28                   Andy Swanson
36                  Mary Kinsella
48                     Structures
73         Manufacturing Research
74     Propulsion-Turbine Engines
75                  Air Armaments
76                    Sustainment
77                        LOM/LOA
78             C4ISR H/W-Aircraft
79                C4ISR H/W-Space
151                  Ken Davidson
171                Kermit Stearns
191             Aircraft Coatings
202                     Stats/DOE
212                        Metals
213                   Soft Matter
214                      Ceramics
215                    Composites
216                          Data
217               Process Control
218                    Simulation
219                    Inspection
220              Electronic Matls
221                Photonic Matls
256                     Airframes
1                         AFRL/RX
23            

In [14]:
#create new NetworkX directed graph
g=nx.Graph()

#add all nodes to graph
for node in nodes:
    g.add_node(node)
    
#iterate through edges and add those
for index, row in raw_data.iterrows():
    g.add_edge(row['Source/Subject'],
               row['Target/Object'],
               key= row['Predicate'],
               weight = row['Weight (1-10)'])

In [15]:
#these are the nodes that are in the raw data that are not in the ontology
set(g.node.keys()).difference(ont_set)

{u'AFRL/RX',
 u'Industry',
 u'Insulation',
 u'Kermit Stearns',
 u'Leading Edge',
 u'Nose Tip',
 u'Steve Szaruga',
 u'Thermal Protection'}

In [16]:
nx.draw_circular(g,with_labels=True,);plt.show()

In [18]:
#list all nodes in the adjacency list, and total number
g.adj.keys(), len(g.adj.keys())

([u'Steve Szaruga',
  u'Soft Matter',
  u'Additive Manufacturing',
  u'Mary Kinsella',
  u'Hypersonic Weapons',
  u'Ken Davidson',
  u'Kermit Stearns',
  u'Air Armaments',
  u'RQ',
  u'Structures',
  u'RX',
  u'C4ISR H/W-Space',
  u'C4ISR H/W-Aircraft',
  u'Thermal Protection',
  u'Airframes',
  u'Data',
  u'Ceramics',
  u'Leading Edge',
  u'Industry',
  u'Process Control',
  u'Andy Swanson',
  u'Simulation',
  u'Sustainment',
  u'RW',
  u'Composites',
  u'Inspection',
  u'Photonic Matls',
  u'LOM/LOA',
  u'Stats/DOE',
  u'Aircraft Coatings',
  u'Propulsion-Turbine Engines',
  u'AFRL/RX',
  u'Metals',
  u'DARPA',
  u'Nose Tip',
  u'Electronic Matls',
  u'Manufacturing Research',
  u'Insulation'],
 38)

In [19]:
#graphs are stored in an adjancey list
g.adj[g.adj.keys()[0]]

{u'AFRL/RX': {'key': u'Belongs In', 'weight': 9},
 u'Additive Manufacturing': {'key': u'Knows', 'weight': 9},
 u'Air Armaments': {'key': u'Knows', 'weight': 1},
 u'Aircraft Coatings': {'key': u'Knows', 'weight': 9},
 u'Airframes': {'key': u'Knows', 'weight': 9},
 u'Andy Swanson': {'key': u'Knows', 'weight': 5},
 u'C4ISR H/W-Aircraft': {'key': u'Knows', 'weight': 1},
 u'C4ISR H/W-Space': {'key': u'Knows', 'weight': 1},
 u'Ceramics': {'key': u'Knows', 'weight': 5},
 u'Composites': {'key': u'Knows', 'weight': 9},
 u'Data': {'key': u'Knows', 'weight': 1},
 u'Electronic Matls': {'key': u'Knows', 'weight': 5},
 u'Hypersonic Weapons': {'key': u'Knows', 'weight': 9},
 u'Inspection': {'key': u'Knows', 'weight': 9},
 u'Ken Davidson': {'key': u'Knows', 'weight': 9},
 u'Kermit Stearns': {'key': u'Knows', 'weight': 9},
 u'LOM/LOA': {'key': u'Knows', 'weight': 9},
 u'Manufacturing Research': {'key': u'Knows', 'weight': 5},
 u'Mary Kinsella': {'key': u'Knows', 'weight': 9},
 u'Metals': {'key': u'Know

In [20]:
g.adj['Additive Manufacturing']

{u'Ceramics': {'key': u'Belongs In', 'weight': 1},
 u'Composites': {'key': u'Belongs In', 'weight': 5},
 u'DARPA': {'key': u'Is Studied By', 'weight': 5},
 u'Data': {'key': u'Belongs In', 'weight': 9},
 u'Electronic Matls': {'key': u'Belongs In', 'weight': 5},
 u'Industry': {'key': u'Is Studied By', 'weight': 9},
 u'Inspection': {'key': u'Belongs In', 'weight': 5},
 u'Mary Kinsella': {'key': u'Knows', 'weight': 9},
 u'Metals': {'key': u'Belongs In', 'weight': 9},
 u'Photonic Matls': {'key': u'Belongs In', 'weight': 1},
 u'Process Control': {'key': u'Belongs In', 'weight': 5},
 u'RQ': {'key': u'Is Studied By', 'weight': 1},
 u'RW': {'key': u'Is Studied By', 'weight': 1},
 u'RX': {'key': u'Is Studied By', 'weight': 9},
 u'Simulation': {'key': u'Belongs In', 'weight': 5},
 u'Soft Matter': {'key': u'Belongs In', 'weight': 9},
 u'Steve Szaruga': {'key': u'Knows', 'weight': 9}}

In [21]:
g.number_of_edges(),g.number_of_nodes(),g.number_of_selfloops()

(276, 38, 0)

In [22]:
g['Steve Szaruga']['Additive Manufacturing']

{'key': u'Knows', 'weight': 9}

In [23]:
g.node['Steve Szaruga']['Label']=set()

In [24]:
g.node['Steve Szaruga']

{'Label': set()}

In [25]:
g.node['Steve Szaruga']['Label'].add('Person')

In [26]:
g.node['Steve Szaruga']

{'Label': {'Person'}}

In [27]:
# how we will check if a node has a label
'Person' in g.node['Steve Szaruga']['Label']

True

In [28]:
g.degree()

{u'AFRL/RX': 1,
 u'Additive Manufacturing': 17,
 u'Air Armaments': 14,
 u'Aircraft Coatings': 14,
 u'Airframes': 15,
 u'Andy Swanson': 8,
 u'C4ISR H/W-Aircraft': 14,
 u'C4ISR H/W-Space': 14,
 u'Ceramics': 21,
 u'Composites': 21,
 u'DARPA': 11,
 u'Data': 21,
 u'Electronic Matls': 19,
 u'Hypersonic Weapons': 16,
 u'Industry': 1,
 u'Inspection': 21,
 u'Insulation': 1,
 u'Ken Davidson': 20,
 u'Kermit Stearns': 20,
 u'LOM/LOA': 14,
 u'Leading Edge': 1,
 u'Manufacturing Research': 14,
 u'Mary Kinsella': 13,
 u'Metals': 21,
 u'Nose Tip': 1,
 u'Photonic Matls': 19,
 u'Process Control': 21,
 u'Propulsion-Turbine Engines': 14,
 u'RQ': 16,
 u'RW': 15,
 u'RX': 25,
 u'Simulation': 21,
 u'Soft Matter': 21,
 u'Stats/DOE': 14,
 u'Steve Szaruga': 27,
 u'Structures': 11,
 u'Sustainment': 14,
 u'Thermal Protection': 1}

In [29]:
node_degrees = pd.DataFrame({'Nodes':g.degree().keys(),'Degree':g.degree().values()})

In [30]:
node_degrees.sort('Degree')

,Degree,Nodes
18,1,Industry
34,1,Nose Tip
31,1,AFRL/RX
17,1,Leading Edge
12,1,Thermal Protection
37,1,Insulation
20,8,Andy Swanson
6,11,Structures
33,11,DARPA
3,13,Mary Kinsella


In [31]:
#add an attribute to each node that contains it log(degree)
for row in node_degrees.iterrows():
    node = row[1]['Nodes']
    g.node[node]['Degree'] = np.log( row[1]['Degree'] )

In [32]:
g.node

{u'AFRL/RX': {'Degree': 0.0},
 u'Additive Manufacturing': {'Degree': 2.8332133440562162},
 u'Air Armaments': {'Degree': 2.6390573296152584},
 u'Aircraft Coatings': {'Degree': 2.6390573296152584},
 u'Airframes': {'Degree': 2.7080502011022101},
 u'Andy Swanson': {'Degree': 2.0794415416798357},
 u'C4ISR H/W-Aircraft': {'Degree': 2.6390573296152584},
 u'C4ISR H/W-Space': {'Degree': 2.6390573296152584},
 u'Ceramics': {'Degree': 3.044522437723423},
 u'Composites': {'Degree': 3.044522437723423},
 u'DARPA': {'Degree': 2.3978952727983707},
 u'Data': {'Degree': 3.044522437723423},
 u'Electronic Matls': {'Degree': 2.9444389791664403},
 u'Hypersonic Weapons': {'Degree': 2.7725887222397811},
 u'Industry': {'Degree': 0.0},
 u'Inspection': {'Degree': 3.044522437723423},
 u'Insulation': {'Degree': 0.0},
 u'Ken Davidson': {'Degree': 2.9957322735539909},
 u'Kermit Stearns': {'Degree': 2.9957322735539909},
 u'LOM/LOA': {'Degree': 2.6390573296152584},
 u'Leading Edge': {'Degree': 0.0},
 u'Manufacturing Re

In [33]:
plt.hist(node_degrees['Degree'].values,range(1,30));plt.show()

In [34]:
np.log(node_degrees['Degree'])

0     3.295837
1     3.044522
2     2.833213
3     2.564949
4     2.772589
5     2.995732
6     2.397895
7     2.639057
8     2.772589
9     2.995732
10    3.218876
11    2.639057
12    0.000000
13    2.639057
14    2.708050
15    3.044522
16    3.044522
17    0.000000
18    0.000000
19    2.639057
20    2.079442
21    3.044522
22    2.639057
23    2.708050
24    3.044522
25    3.044522
26    2.944439
27    2.639057
28    2.639057
29    2.639057
30    2.639057
31    0.000000
32    3.044522
33    2.397895
34    0.000000
35    2.944439
36    3.044522
37    0.000000
Name: Degree, dtype: float64

In [35]:
#attributes of nodes are stored in teh g.node dictionary
g.node

{u'AFRL/RX': {'Degree': 0.0},
 u'Additive Manufacturing': {'Degree': 2.8332133440562162},
 u'Air Armaments': {'Degree': 2.6390573296152584},
 u'Aircraft Coatings': {'Degree': 2.6390573296152584},
 u'Airframes': {'Degree': 2.7080502011022101},
 u'Andy Swanson': {'Degree': 2.0794415416798357},
 u'C4ISR H/W-Aircraft': {'Degree': 2.6390573296152584},
 u'C4ISR H/W-Space': {'Degree': 2.6390573296152584},
 u'Ceramics': {'Degree': 3.044522437723423},
 u'Composites': {'Degree': 3.044522437723423},
 u'DARPA': {'Degree': 2.3978952727983707},
 u'Data': {'Degree': 3.044522437723423},
 u'Electronic Matls': {'Degree': 2.9444389791664403},
 u'Hypersonic Weapons': {'Degree': 2.7725887222397811},
 u'Industry': {'Degree': 0.0},
 u'Inspection': {'Degree': 3.044522437723423},
 u'Insulation': {'Degree': 0.0},
 u'Ken Davidson': {'Degree': 2.9957322735539909},
 u'Kermit Stearns': {'Degree': 2.9957322735539909},
 u'LOM/LOA': {'Degree': 2.6390573296152584},
 u'Leading Edge': {'Degree': 0.0},
 u'Manufacturing Re

In [36]:
#attributes of edges are stored in the g.edges dictionary
g.edge

{u'AFRL/RX': {u'Steve Szaruga': {'key': u'Belongs In', 'weight': 9}},
 u'Additive Manufacturing': {u'Ceramics': {'key': u'Belongs In', 'weight': 1},
  u'Composites': {'key': u'Belongs In', 'weight': 5},
  u'DARPA': {'key': u'Is Studied By', 'weight': 5},
  u'Data': {'key': u'Belongs In', 'weight': 9},
  u'Electronic Matls': {'key': u'Belongs In', 'weight': 5},
  u'Industry': {'key': u'Is Studied By', 'weight': 9},
  u'Inspection': {'key': u'Belongs In', 'weight': 5},
  u'Mary Kinsella': {'key': u'Knows', 'weight': 9},
  u'Metals': {'key': u'Belongs In', 'weight': 9},
  u'Photonic Matls': {'key': u'Belongs In', 'weight': 1},
  u'Process Control': {'key': u'Belongs In', 'weight': 5},
  u'RQ': {'key': u'Is Studied By', 'weight': 1},
  u'RW': {'key': u'Is Studied By', 'weight': 1},
  u'RX': {'key': u'Is Studied By', 'weight': 9},
  u'Simulation': {'key': u'Belongs In', 'weight': 5},
  u'Soft Matter': {'key': u'Belongs In', 'weight': 9},
  u'Steve Szaruga': {'key': u'Knows', 'weight': 9}},


In [37]:
#the edge between the "Air Armaments" and "Simulation" nodes is labeled 'Involves' and has the attribute weight:1
g.edge['Air Armaments']['Simulation']

{'key': u'Involves', 'weight': 1}

In [38]:
#explore connecting ontology to labeling of nodes
for node in g.node:
    g.node[node]['Label'] = []
    for key in ontology.keys():
        if node in ontology[key].values:
            g.node[node]['Label'].append(key)
    print node, g.node[node]

Steve Szaruga {'Degree': 3.2958368660043291, 'Label': []}
Soft Matter {'Degree': 3.044522437723423, 'Label': [u'RX Function']}
Additive Manufacturing {'Degree': 2.8332133440562162, 'Label': [u'Discliplines']}
Mary Kinsella {'Degree': 2.5649493574615367, 'Label': [u'Contact']}
Hypersonic Weapons {'Degree': 2.7725887222397811, 'Label': [u'Discliplines']}
Ken Davidson {'Degree': 2.9957322735539909, 'Label': [u'Contact']}
Kermit Stearns {'Degree': 2.9957322735539909, 'Label': []}
Air Armaments {'Degree': 2.6390573296152584, 'Label': [u'RXM Roadmap']}
RQ {'Degree': 2.7725887222397811, 'Label': [u'Contact']}
Structures {'Degree': 2.3978952727983707, 'Label': [u'RXM Roadmap']}
RX {'Degree': 3.2188758248682006, 'Label': [u'Contact']}
C4ISR H/W-Space {'Degree': 2.6390573296152584, 'Label': [u'RXM Roadmap']}
C4ISR H/W-Aircraft {'Degree': 2.6390573296152584, 'Label': [u'RXM Roadmap']}
Thermal Protection {'Degree': 0.0, 'Label': []}
Airframes {'Degree': 2.7080502011022101, 'Label': [u'Discliplines

In [39]:
g.node

{u'AFRL/RX': {'Degree': 0.0, 'Label': []},
 u'Additive Manufacturing': {'Degree': 2.8332133440562162,
  'Label': [u'Discliplines']},
 u'Air Armaments': {'Degree': 2.6390573296152584, 'Label': [u'RXM Roadmap']},
 u'Aircraft Coatings': {'Degree': 2.6390573296152584,
  'Label': [u'Discliplines']},
 u'Airframes': {'Degree': 2.7080502011022101, 'Label': [u'Discliplines']},
 u'Andy Swanson': {'Degree': 2.0794415416798357, 'Label': [u'Contact']},
 u'C4ISR H/W-Aircraft': {'Degree': 2.6390573296152584,
  'Label': [u'RXM Roadmap']},
 u'C4ISR H/W-Space': {'Degree': 2.6390573296152584, 'Label': [u'RXM Roadmap']},
 u'Ceramics': {'Degree': 3.044522437723423, 'Label': [u'RX Function']},
 u'Composites': {'Degree': 3.044522437723423, 'Label': [u'RX Function']},
 u'DARPA': {'Degree': 2.3978952727983707, 'Label': [u'Contact']},
 u'Data': {'Degree': 3.044522437723423, 'Label': [u'RX Function']},
 u'Electronic Matls': {'Degree': 2.9444389791664403,
  'Label': [u'RX Function']},
 u'Hypersonic Weapons': {'De

In [40]:
import json
from networkx.readwrite import json_graph
data = json_graph.node_link_data(g)
with open('graph.json', 'w') as f:
    json.dump(data, f, indent=4)

In [41]:
%%html
<div id="d3-example"></div>
<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [42]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.
    
    // First, we specify the size of the canvas containing
    // the visualization (size of the <div> element).
    var width = 800,
        height = 600;

    // We create a color scale.
    var color = d3.scale.category10();

    // We create a force-directed dynamic graph layout.
    var force = d3.layout.force()
        .charge(-120)
        .linkDistance(30)
        .size([width, height]);

    // In the <div> element, we create a <svg> graphic
    // that will contain our interactive visualization.
    var svg = d3.select("#d3-example").select("svg")
    if (svg.empty()) {
        svg = d3.select("#d3-example").append("svg")
                    .attr("width", width)
                    .attr("height", height);
    }
    
    // create the zoom listener
    var zoomListener = d3.behavior.zoom()
      .scaleExtent([0.1, 3])
      .on("zoom", zoomHandler);

    // function for handling zoom event
    function zoomHandler() {
      vis.attr("transform", "translate(" + d3.event.translate + ")scale(" + d3.event.scale + ")");
    }

    
    
    // We load the JSON file.
    d3.json("graph.json", function(error, graph) {
        // In this block, the file has been loaded
        // and the 'graph' object contains our graph.
        
        // We load the nodes and links in the force-directed
        // graph.
        force.nodes(graph.nodes)
            .links(graph.links)
            .start();

        // We create a <line> SVG element for each link
        // in the graph.
        var link = svg.selectAll(".link")
            .data(graph.links)
            .enter().append("line")
            .attr("class", "link");

        // We create a <circle> SVG element for each node
        // in the graph, and we specify a few attributes.
        var node = svg.selectAll(".node")
            .data(graph.nodes)
            .enter().append("circle")
            .attr("class", "node")
            .attr("r", function(d){return 2*d.Degree+4})  // radius
            .style("fill", function(d) {
                // The node color depends on the club.
                return color(d.Label[0]); 
            })
            .call(force.drag);

        // The name of each node is the node number.
        node.append("title")
            .text(function(d) { return d.id; });

        // We bind the positions of the SVG elements
        // to the positions of the dynamic force-directed graph,
        // at each time step.
        force.on("tick", function() {
            link.attr("x1", function(d) { return d.source.x; })
                .attr("y1", function(d) { return d.source.y; })
                .attr("x2", function(d) { return d.target.x; })
                .attr("y2", function(d) { return d.target.y; });

            node.attr("cx", function(d) { return d.x; })
                .attr("cy", function(d) { return d.y; });
        });
        
        // apply the zoom behavior to the svg image
        zoomListener(svg);
        
    });
});

<IPython.core.display.Javascript object>

# networkx to javascript explroation 

In [15]:
from IPython.display import Javascript
Javascript("element.append('Hello World, Again!');")

<IPython.core.display.Javascript object>

In [17]:
%%javascript
require.config({
    paths: {
        'three': "http://cdnjs.cloudflare.com/ajax/libs/three.js/r69/three",
    }
});

<IPython.core.display.Javascript object>

In [20]:
%%javascript
require(['three'], function() {
    element.append(THREE.REVISION);
});

<IPython.core.display.Javascript object>

In [ ]:
%%javascript
var renderer = new THREE.WebGLRenderer({ antialias: true });
var canvas = renderer.domElement;
element.append(canvas);
var camera = new THREE.PerspectiveCamera(70, canvas.width / canvas.height, 1, 1000);
camera.position.z = 400;
var scene = new THREE.Scene();
var material = new THREE.MeshDepthMaterial();
var mesh = new THREE.Mesh(new THREE.CubeGeometry(200, 200, 200), material);
scene.add(mesh);
function animate() {
    renderer.render(scene, camera);
    mesh.rotation.x += 0.005;
    mesh.rotation.y += 0.01;
    requestAnimationFrame(animate);
}
animate();

%%javascript
require(['three'], function() {
    element.append(THREE.REVISION);
});

Generating Javascript from Python
You often want to generate Javascript from Python.

Here's a hack that takes a fragment of Javascript, finds all the functions that are declared, and generates bound Python methods for each function:

In [24]:
import types
from functools import partial
import json

class JSCode(object):
    def __init__(self, code):
        self.code = code
        self.invokes = ""
        
        # Find all the function name(  lines in the Javasript code
        for line in code.split("\n"):
            line = line.strip().split()
            if line and line[0] == 'function':
                funcname = line[1].split("(")[0]
                
                # For each create a bound method to add to the list of invocations
                def invoke(self, *args):
                    argstr = ','.join([json.dumps(arg) for arg in args])
                    self.invokes += "%s(%s);\n" % (funcname, argstr)                

                setattr(self, funcname, types.MethodType(invoke, self))    
                
    def _repr_javascript_(self):
        """Return the code definitions and all invocations"""
        return self.code + self.invokes

In [25]:
code = JSCode("""
    function test() {
        console.log("test!");
    }

    function output(string) {
        element.append(string);
    }
""")

Now, the code object has test() and output() methods corresponding to Javascript methods:

In [26]:
code.output("foo")
code.output(" and bar")
code

Here is the corresponding Javascript fragment that was generated:

In [27]:
print code._repr_javascript_()


    function test() {
        console.log("test!");
    }

    function output(string) {
        element.append(string);
    }
output("foo");
output(" and bar");

